In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import meshio

In [11]:
class edge:
    def __init__(self, node0, node1):
        self.composite  = False # is the edge made of subedges?
        # list indexes of nodes
        self.nodes = [node0, node1]
        # lis of indexes of sub edges
        self.subedges = [0, 0]
    
class element:
    def __init__(self, edge0, edge1, edge2):
        self.composite = False
        # list of indexes of edges
        self.edges = [edge0, edge1, edge2]
        # list of indexes of sub_elems
        self.subelems = [0, 0, 0, 0]
        
class mesh:
    def __init__(self, elements, edges, nodes):
        self.elements = elements # list of type element
        self.edges = edges # list of type edge
        self.nodes = nodes # list of 2D coordinates (np.array)

In [14]:
def load_mesh(filename):
    """
    Loads and parses a 2D mesh in msh format

    Parameters:
    - filename: path to the mesh file

    Returns:
    - Mesh object
    """
    # 1) LOAD RAW MESH:
    mesh_file_path = filename
    raw_mesh = meshio.read(mesh_file_path)
    # raw face2node connectivity
    raw_elements = raw_mesh.cells[0].data
    raw_elements = raw_elements.astype(int)
    # raw node coordinates
    raw_coords = raw_mesh.points
    # 2) CREATE A RECURSIVE MESH
    # 2_1) initializations
    elements = [] # to fill with type element
    edges = [] # to fill with type edge
    nodes = [] # to fill with 2D np. arrays
    n_elements = 0
    n_edges = 0
    n_nodes = 0
    # 2_2) loop that creates elements and edges
    for iel in range(raw_elements.shape[0]):
        idx_edges = [0, 0, 0]
        for ie in range(3):
            node_0 = raw_elements[iel,ie]
            node_1 = raw_elements[iel,(ie+1)%3]
            [not_to_add, idx_edge] = edge_already_there(node_0,node_1, edges)
            if (not_to_add):
                # register index of existing edge
                idx_edges[ie] = idx_edge
            else:
                # create a new edge
                new_edge = edge(node_0,node_1)
                # add edge to list
                edges.append(new_edge)
                n_edges = n_edges +1
                # index of the edge is size of the list
                idx_edges[ie] = n_edges
        # create new element 
        new_element = element(idx_edges[0], idx_edges[1], idx_edges[2]) 
        # add it to list
        elements.append(new_element)
        n_elements = n_elements + 1
    # 2_3) loop to create nodes
    for ino in range (raw_coords.shape[0]):
        # create node
        new_node= np.array(raw_coords[ino,:])
        # add it to list
        nodes.append(new_node)
        n_node = n_nodes+1 # useless counter
    return mesh(elements, edges, nodes)

def edge_already_there(node_0,node_1,edges):
    if (len(edges)==0):
        return [False,0]
    for (ie,edge) in enumerate(edges):
        start = edge.nodes[0]
        end = edge.nodes[1]
        if ((start == node_0 and end == node_1) or (start == node_1 and end == node_0)):
            return [True, ie]
        else:
            return [False, 0]

In [15]:
mesh_file_path = "square_L1_N5.msh"
mesh = load_mesh(mesh_file_path)

In [16]:
def split_edge(mesh,ie):
    edge = mesh.edges[ie]
    if (not edge.composite):
        # Create and add new point
        coord_0 = mesh.nodes[edge.nodes[0]]
        coord_1 = mesh.nodes[edge.nodes[1]]
        new_coord = 0.5*(coord_0+coord_1)
        no_nodes = len(mesh.nodes)
        mesh.nodes.append(new_coord)
        # create and add new edges
        new_edge_0 = edge(edge.nodes[0], no_nodes+1)
        new_edge_1 = edge(no_nodes+1, edge.nodes[1])
        no_edges = len(mesh.edges)
        mesh.edges.append(new_edge_0)
        mesh.edges.append(new_edge_1)
        # edge is now composite
        edge.composite = True
        edge.subedges[0] = no_edges
        edge.subedges[1] = no_edges + 1
        
def split_element(mesh, iel):
    element = mesh.elements[iel]
    if (not element.composite):
        # midpoints (to create new edges)
        midpoints = []
        # idx of side edges (to create new elems)
        idx_side_edges = element.edges
        # loop to split edges
        for ie in range(3):
            idx_edge = element.edges[ie]
            idx_side_edges[ie] = idx_edge 
            edge = mesh.edges[idx_edge]
            # split edge if not composite
            if (not edge.composite):
                split_edge(mesh, element.edges[ie])
            midpoints[ie] = edge.subedges[0].nodes[0]
        # loop to create new edges (internal to element)
        idx_new_edges = []
        no_edges = len(mesh.edges)
        for ie in range(3):
            new_edge = edge(midpoints[ie], midpoints[(ie+1)%3])
            mesh.edges.append(new_edge)
            idx_new_edges.append(no_edges)
            no_edges = no_edges +1
        # get necessary edges 
        side_edge_0 = mesh.edges[idx_side_edges[0]]
        side_edge_1 = mesh.edges[idx_side_edges[1]]
        side_edge_2 = mesh.edges[idx_side_edges[2]]
        # create and add new subelems
        no_elems_old = len(mesh.elements)
        new_elem_0 = element(side_edge_0.subedges[0], idx_new_edges[2], side_edge_2.subedges[1])
        mesh.elements.append(new_elem_0)
        new_elem_1 = element(side_edge_1.subedges[0], idx_new_edges[0], side_edge_0.subedges[1])
        mesh.elements.append(new_elem_1)
        new_elem_2 = element(side_edge_2.subedges[0], idx_new_edges[1], side_edge_1.subedges[1])
        mesh.elements.append(new_elem_2)
        new_elem_3 = element(idx_new_edges[0], idx_new_edges[1], idx_new_edges[2])
        mesh.elements.append(new_elem_3)
        # element is now composite
        element.composite = True
        element.subelems[0] = no_elems_old  
        element.subelems[1] = no_elems_old+1
        element.subelems[2] = no_elems_old+2 
        element.subelems[3] = no_elems_old+3